# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os
import tqdm

#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [2]:
data_dir = '../train'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

#### Determine if an nvidia GPU is available

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [4]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [5]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

# for i, (x, y) in tqdm.tqdm(enumerate(loader)):
#     mtcnn(x, save_path=y)
#     print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
# del mtcnn

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [6]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

In [7]:
resnet.state_dict().keys()

odict_keys(['conv2d_1a.conv.weight', 'conv2d_1a.bn.weight', 'conv2d_1a.bn.bias', 'conv2d_1a.bn.running_mean', 'conv2d_1a.bn.running_var', 'conv2d_1a.bn.num_batches_tracked', 'conv2d_2a.conv.weight', 'conv2d_2a.bn.weight', 'conv2d_2a.bn.bias', 'conv2d_2a.bn.running_mean', 'conv2d_2a.bn.running_var', 'conv2d_2a.bn.num_batches_tracked', 'conv2d_2b.conv.weight', 'conv2d_2b.bn.weight', 'conv2d_2b.bn.bias', 'conv2d_2b.bn.running_mean', 'conv2d_2b.bn.running_var', 'conv2d_2b.bn.num_batches_tracked', 'conv2d_3b.conv.weight', 'conv2d_3b.bn.weight', 'conv2d_3b.bn.bias', 'conv2d_3b.bn.running_mean', 'conv2d_3b.bn.running_var', 'conv2d_3b.bn.num_batches_tracked', 'conv2d_4a.conv.weight', 'conv2d_4a.bn.weight', 'conv2d_4a.bn.bias', 'conv2d_4a.bn.running_mean', 'conv2d_4a.bn.running_var', 'conv2d_4a.bn.num_batches_tracked', 'conv2d_4b.conv.weight', 'conv2d_4b.bn.weight', 'conv2d_4b.bn.bias', 'conv2d_4b.bn.running_mean', 'conv2d_4b.bn.running_var', 'conv2d_4b.bn.num_batches_tracked', 'repeat_1.0.bran

In [10]:
#freeze layers
for param in resnet.parameters():
    param.requires_grad = False

resnet.last_bn.weight.requires_grad = True
resnet.last_bn.bias.requires_grad = True
resnet.last_linear.weight.requires_grad = True

    

#### Define optimizer, scheduler, dataset, and dataloader

In [13]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

#### Define loss and evaluation functions

In [14]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [15]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |    59/59   | loss:    3.4289 | fps:   17.6495 | acc:    0.0387   

Epoch 1/8
----------
Train |   234/234  | loss:    0.6190 | fps:   16.4215 | acc:    0.8626   
Valid |    59/59   | loss:    0.3887 | fps:   17.6030 | acc:    0.9231   

Epoch 2/8
----------
Train |   234/234  | loss:    0.4518 | fps:   16.4594 | acc:    0.9040   
Valid |    59/59   | loss:    0.3615 | fps:   17.9722 | acc:    0.9205   

Epoch 3/8
----------
Train |   234/234  | loss:    0.4141 | fps:   16.1911 | acc:    0.9116   
Valid |    13/59   | loss:    0.3912 | fps:   15.4112 | acc:    0.9159   

In [ ]:
#save model
torch.save(resnet.state_dict(), 'resnetmodel_finetune.pt')